In [1]:
import os, json

from others.utils import read_airland_file, generate_separation_between_runways, save_solution
from models.MIP import solve_single_runway_mip, solve_multiple_runways_mip
from models.CP import solve_single_runway_cp, solve_multiple_runways_cp
from models.CP_MIP import solve_hybrid_lbbd
from ortools.sat.python import cp_model

In [2]:
data_dir = "data"
r_max = 5
num_files = 8
output_json = "results/metrics.json"
solutions_json = "results/solutions.json"

# Results

### Justificação para o report

**Dataset 9** represents a critical scalability threshold for the MIP formulation of the aircraft landing problem. With **100 aircraft** and a **large freeze-time (720)**, the model already exhibits a substantial increase in both the number of decision variables and constraints. This growth is mainly driven by the quadratic expansion of binary sequencing variables. From this dataset onward, the computational burden increases dramatically, clearly highlighting the practical limitations of exact MIP approaches for large-scale instances.

#### MIP Model Size per Dataset

| Dataset | Number of Aircraft | Decision Variables | Constraints | Estimated Runtime |
| ------: | -----------------: | -----------------: | ----------: | ----------------: |
|       9 |                100 |             10,200 |      12,019 |              Days |
|      10 |                150 |             22,800 |      25,569 |      Days / Weeks |
|      11 |                200 |             40,400 |      44,184 |    Weeks / Months |
|      12 |                250 |             63,000 |      67,977 |    Months / Years |
|      13 |                500 |            251,000 |     261,552 |       Impractical |

The computational results clearly show the exponential growth in complexity of the exact MIP formulation for the aircraft landing problem. While instances with up to 50 aircraft can be solved to optimality within seconds, doubling the number of aircraft to 100 already leads to solution times of several hours, especially under large freeze-time windows (e.g., 720). This behaviour is explained by the quadratic number of binary sequencing variables and the resulting exponential branch-and-bound search space. Based on empirical evidence and theoretical complexity, instances with 150 or more aircraft become computationally intractable for exact MIP solvers, with expected solution times ranging from days to months or even longer. Therefore, large-scale instances are unsuitable for exact optimization and must instead be addressed using time-limited approaches, heuristic methods, or alternative formulations.


## MIP

### Single Runway

In [3]:
if os.path.exists(output_json):
    with open(output_json, "r") as f:
        all_metrics = json.load(f)
else:
    all_metrics = {}

all_metrics["MIP Single"] = []

for i in range(1, num_files + 1):
    file_name = f"airland{i}.txt"
    file_path = os.path.join(data_dir, file_name)

    print("\n" + "="*60)
    print(f"Processing file: {file_name}")
    print("="*60 + "\n")

    data = read_airland_file(file_path)

    solver, variables, metrics = solve_single_runway_mip(data['p'], data['planes'], data['separation_times'], hint=True, performance=True)

    save_solution(
        solver = solver,
        variables=variables,
        num_planes=data['p'],
        data=data['planes'],
        solution_file=solutions_json,
        tag="MIP Single",
        dataset_name=file_name
    )

    metrics_with_file = {"file": file_name, **metrics}
    all_metrics["MIP Single"].append(metrics_with_file)

with open(output_json, "w") as f:
    json.dump(all_metrics, f, indent=4)

print(f"\n-> Metrics saved to {output_json}")

# Metrics Table
print("\nMetrics Table:")
print("{:<12} {:<15} {:<15} {:<15} {:<15} {:<15}".format(
    "File", "Exec Time(s)", "Variables", "Constraints", "Total Penalty", "B&B Nodes"
))
for m in all_metrics["MIP Single"]:
    print("{:<12} {:<15.4f} {:<15} {:<15} {:<15} {:<15}".format(
        m["file"], m["execution_time"], m["num_variables"],
        m["num_constraints"], m["total_penalty"], m["num_branch_and_bound_nodes"]
    ))



Processing file: airland1.txt

		Creating Single Runway MIP Model

-> Decision variables: 120
-> Constraints: 205

			Solving MIP

-> Landing times of all planes:
Plane | Landing Time | Earliest | Target | Latest
-------------------------------------------------
    0 |       165.00 |   129.00 | 155.00 | 559.00
    1 |       258.00 |   195.00 | 258.00 | 744.00
    2 |        98.00 |    89.00 |  98.00 | 510.00
    3 |       106.00 |    96.00 | 106.00 | 521.00
    4 |       118.00 |   110.00 | 123.00 | 555.00
    5 |       126.00 |   120.00 | 135.00 | 576.00
    6 |       134.00 |   124.00 | 138.00 | 577.00
    7 |       142.00 |   126.00 | 140.00 | 573.00
    8 |       150.00 |   135.00 | 150.00 | 591.00
    9 |       180.00 |   160.00 | 180.00 | 657.00

-> Planes that did not land on the target time:
Plane | Landing Time | Target | Early Dev | Late Dev | Penalty
--------------------------------------------------------------
    0 |       165.00 | 155.00 |      0.00 |    10.00 |  100.0

### Multiple Runways

In [4]:
if os.path.exists(output_json):
    with open(output_json, "r") as f:
        all_metrics = json.load(f)
else:
    all_metrics = {}

all_metrics["MIP Multiple"] = []

for i in range(1, num_files + 1):
    file_name = f"airland{i}.txt"
    file_path = os.path.join(data_dir, file_name)

    print("\n" + "="*60)
    print(f"Processing file: {file_name}")
    print("="*60 + "\n")

    data = read_airland_file(file_path)

    for r in range(1, r_max + 1):
        print(f"\n---> Solving with {r} runways...\n")
        separation_times_between_runways = generate_separation_between_runways(data['p'], r, separation_same_runway=True, default_between_runways=2)
        solver, variables, metrics = solve_multiple_runways_mip(data['p'], r, data['planes'], data['separation_times'], separation_times_between_runways,
                                                   hint=True, performance=True)

        penalty = metrics["total_penalty"]
        print(f"Runways={r} -> Total Penalty={penalty}")

        save_solution(
            solver = solver,
            variables=variables,
            num_planes=data['p'],
            data=data['planes'],
            solution_file=solutions_json,
            tag="MIP Multiple",
            dataset_name=file_name,
            num_runways=r
        )

        metrics_with_file = {"file": file_name, "num_runways": r, **metrics}
        all_metrics["MIP Multiple"].append(metrics_with_file)

        # Interrupt if zero penalty is found
        if penalty == 0:
            break

with open(output_json, "w") as f:
    json.dump(all_metrics, f, indent=4)

print(f"\n-> Metrics saved to {output_json}")

# Metrics Table
print("\nMetrics Table (MIP Multiple):")
header = "{:<12} {:<18} {:<15} {:<15} {:<15} {:<15} {:<15}".format(
    "File", "Number of Runways", "Exec Time(s)", "Variables", "Constraints", "Total Penalty", "B&B Nodes"
)
print(header)
print("-" * len(header))

for m in all_metrics["MIP Multiple"]:
    print("{:<12} {:<18} {:<15.4f} {:<15} {:<15} {:<15} {:<15}".format(
        m["file"], m["num_runways"], m["execution_time"], m["num_variables"],
        m["num_constraints"], m["total_penalty"], m["num_branch_and_bound_nodes"]
    ))



Processing file: airland1.txt


---> Solving with 1 runways...

		Creating Multiple Runways MIP Solver

-> Decision variables: 220
-> Constraints: 305

			Solving MIP

-> Landing times of all planes:
Plane | Landing Time | Earliest | Target | Latest | Runway
----------------------------------------------------------
    0 |       165.00 |   129.00 | 155.00 | 559.00 |      0
    1 |       258.00 |   195.00 | 258.00 | 744.00 |      0
    2 |        98.00 |    89.00 |  98.00 | 510.00 |      0
    3 |       106.00 |    96.00 | 106.00 | 521.00 |      0
    4 |       118.00 |   110.00 | 123.00 | 555.00 |      0
    5 |       126.00 |   120.00 | 135.00 | 576.00 |      0
    6 |       134.00 |   124.00 | 138.00 | 577.00 |      0
    7 |       142.00 |   126.00 | 140.00 | 573.00 |      0
    8 |       150.00 |   135.00 | 150.00 | 591.00 |      0
    9 |       180.00 |   160.00 | 180.00 | 657.00 |      0

-> Planes that did not land on the target time:
Plane | Landing Time | Target | Early Dev 

### Large Datasets

In [ ]:
if os.path.exists(output_json):
    with open(output_json, "r") as f:
        all_metrics = json.load(f)
else:
    all_metrics = {}

all_metrics["MIP Multiple Large Datasets"] = []

# Values of runways to test for each dataset, based on prior knowledge
runways = {'9': [3,4], '10': [4,5], '11': [5], '12': [6], '13': [7]}

for i in range(9, 14):
    file_name = f"airland{i}.txt"
    file_path = os.path.join(data_dir, file_name)

    print("\n" + "="*60)
    print(f"Processing file: {file_name}")
    print("="*60 + "\n")

    data = read_airland_file(file_path)

    for r in runways[str(i)]:
        print(f"\n---> Solving with {r} runways...\n")
        separation_times_between_runways = generate_separation_between_runways(data['p'], r, separation_same_runway=True, default_between_runways=2)
        solver, variables, metrics = solve_multiple_runways_mip(data['p'], r, data['planes'], data['separation_times'], separation_times_between_runways,
                                                   hint=True, performance=True)

        penalty = metrics["total_penalty"]
        print(f"Runways={r} -> Total Penalty={abs(penalty):.2f}")

        save_solution(
            solver = solver,
            variables=variables,
            num_planes=data['p'],
            data=data['planes'],
            solution_file=solutions_json,
            tag="MIP Multiple Large Datasets",
            dataset_name=file_name,
            num_runways=r
        )

        metrics_with_file = {"file": file_name, "num_runways": r, **metrics}
        all_metrics["MIP Multiple Large Datasets"].append(metrics_with_file)

        # Interrupt if zero penalty is found
        if penalty == 0:
            break

with open(output_json, "w") as f:
    json.dump(all_metrics, f, indent=4)

print(f"\n-> Metrics saved to {output_json}")

# Metrics Table
print("\nMetrics Table (MIP Multiple Large Datasets):")
header = "{:<12} {:<18} {:<15} {:<15} {:<15} {:<15} {:<15}".format(
    "File", "Number of Runways", "Exec Time(s)", "Variables", "Constraints", "Total Penalty", "B&B Nodes"
)
print(header)
print("-" * len(header))

for m in all_metrics["MIP Multiple Large Datasets"]:
    print("{:<12} {:<18} {:<15.4f} {:<15} {:<15} {:<15} {:<15}".format(
        m["file"], m["num_runways"], m["execution_time"], m["num_variables"],
        m["num_constraints"], abs(m["total_penalty"]), m["num_branch_and_bound_nodes"]
    ))



Processing file: airland9.txt


---> Solving with 3 runways...

		Creating Multiple Runways MIP Solver

-> Decision variables: 20400
-> Constraints: 31919

			Solving MIP

-> Landing times of all planes:
Plane | Landing Time | Earliest |   Target |   Latest | Runway
--------------------------------------------------------------
    0 |       908.00 |   601.00 |   908.00 |  2401.00 |      2
    1 |      1027.00 |   725.00 |  1027.00 |  2525.00 |      0
    2 |      1378.00 |   928.00 |  1378.00 |  2728.00 |      1
    3 |      1502.00 |   974.00 |  1502.00 |  2774.00 |      2
    4 |      1618.00 |  1056.00 |  1618.00 |  2856.00 |      2
    5 |      1235.00 |  1119.00 |  1235.00 |  2919.00 |      1
    6 |      1520.00 |  1148.00 |  1520.00 |  2948.00 |      0
    7 |      1401.00 |  1195.00 |  1401.00 |  2995.00 |      0
    8 |      1584.00 |  1201.00 |  1584.00 |  3001.00 |      1
    9 |      1343.00 |  1218.00 |  1343.00 |  3018.00 |      2
   10 |      1787.00 |  1224.00 |  1787

## CP

### Single Runway

In [3]:
if os.path.exists(output_json):
    with open(output_json, "r") as f:
        all_metrics = json.load(f)
else:
    all_metrics = {}

all_metrics["CP Single"] = []

strategies = {
    "Automatic Search": cp_model.AUTOMATIC_SEARCH,
    "Fixed Search": cp_model.FIXED_SEARCH,
    "Portfolio Search": cp_model.PORTFOLIO_SEARCH,
    "LP Search": cp_model.LP_SEARCH
}

for name, strategy in strategies.items():
    print(f"\n=== Testing strategy: {name} ===\n")
    tag = f"CP Single - Strategy {name}"

    for i in range(1, num_files + 1):
        file_name = f"airland{i}.txt"
        file_path = os.path.join(data_dir, file_name)

        print("\n" + "="*60)
        print(f"Processing file: {file_name}")
        print("="*60 + "\n")

        data = read_airland_file(file_path)

        solver, model, variables, metrics = solve_single_runway_cp(
            data['p'], data['planes'], data['separation_times'], hint=True, search_strategy=strategy, performance=True)

        save_solution(
            solver = solver,
            variables=variables,
            num_planes=data['p'],
            data=data['planes'],
            solution_file=solutions_json,
            tag = tag,
            dataset_name=file_name
        )

        metrics_with_file = {"file": file_name, "strategy": name, **metrics}
        all_metrics["CP Single"].append(metrics_with_file)

with open(output_json, "w") as f:
    json.dump(all_metrics, f, indent=4)

print(f"\n-> Metrics saved to {output_json}")

# Metrics Table
print("\nMetrics Table (CP Single):")
print("{:<12} {:<18} {:<18} {:<18} {:<18} {:<18} {:<18} {:<18} {:<18} {:<18} {:<18}".format(
    "File", "Strategy", "Exec Time(s)", "Memory Usage", "Status", "Conflicts", "Branches", "Booleans", "Best Objective Bound", "Variables", "Constraints"
))
for m in all_metrics["CP Single"]:
    print("{:<12} {:<18} {:<18.4f} {:<18} {:<18} {:<18} {:<18} {:<18} {:<18} {:<18} {:<18}".format(
        m["file"], m["strategy"], m["execution_time"], m["memory_usage"],
        m["solution_status"], m["num_conflicts"], m["num_branches"],
        m["num_booleans"], m["best_objective_bound"], m["num_variables"], m["num_constraints"]
    ))


=== Testing strategy: Automatic Search ===


Processing file: airland1.txt

		     Creating CP model

-> Number of decision variables created: 75
-> Number of constraints: 130

			Solving CP

-> Landing times of all planes:
Plane | Landing Time | Earliest | Target | Latest
-------------------------------------------------
    0 |       165.00 |   129.00 | 155.00 | 559.00
    1 |       258.00 |   195.00 | 258.00 | 744.00
    2 |        98.00 |    89.00 |  98.00 | 510.00
    3 |       106.00 |    96.00 | 106.00 | 521.00
    4 |       118.00 |   110.00 | 123.00 | 555.00
    5 |       126.00 |   120.00 | 135.00 | 576.00
    6 |       134.00 |   124.00 | 138.00 | 577.00
    7 |       142.00 |   126.00 | 140.00 | 573.00
    8 |       150.00 |   135.00 | 150.00 | 591.00
    9 |       180.00 |   160.00 | 180.00 | 657.00

-> Planes that did not land on the target time:
Plane | Landing Time | Target | Early Dev | Late Dev | Penalty
--------------------------------------------------------------


### Multiple Runway

In [4]:
if os.path.exists(output_json):
    with open(output_json, "r") as f:
        all_metrics = json.load(f)
else:
    all_metrics = {}

all_metrics["CP Multiple"] = []

strategies = {
    "Automatic Search": cp_model.AUTOMATIC_SEARCH,
    "Fixed Search": cp_model.FIXED_SEARCH,
    "Portfolio Search": cp_model.PORTFOLIO_SEARCH,
    "LP Search": cp_model.LP_SEARCH
}

for name, strategy in strategies.items():
    print(f"\n=== Testing strategy: {name} ===\n")
    tag = f"CP Multiple - Strategy {name}"

    for i in range(1, num_files + 1):
        file_name = f"airland{i}.txt"
        file_path = os.path.join(data_dir, file_name)

        print("\n" + "="*60)
        print(f"Processing file: {file_name}")
        print("="*60 + "\n")

        data = read_airland_file(file_path)

        for r in range(1, r_max + 1):
            print(f"\n---> Solving with {r} runways...\n")
            separation_times_between_runways = generate_separation_between_runways(data['p'], r, separation_same_runway=True, default_between_runways=2)
            solver, model, variables, metrics = solve_multiple_runways_cp(data['p'], r, data['planes'], data['separation_times'],
                                                                separation_times_between_runways, hint=True, search_strategy=strategy, performance=True)

            penalty = metrics["best_objective_bound"]
            print(f"Runways={r} -> Best Objective Bound={penalty}")

            save_solution(
                solver = solver,
                variables=variables,
                num_planes=data['p'],
                data=data['planes'],
                solution_file=solutions_json,
                tag=tag,
                dataset_name=file_name,
                num_runways=r
            )

            metrics_with_file = {"file": file_name, "strategy": name, "num_runways": r, **metrics}
            all_metrics["CP Multiple"].append(metrics_with_file)

            # Interrupt if zero penalty is found
            if penalty == 0:
                break

with open(output_json, "w") as f:
    json.dump(all_metrics, f, indent=4)

print(f"\n-> Metrics saved to {output_json}")

# Metrics Table
print("\nMetrics Table (CP Multiple):")
print("{:<12} {:<18} {:<18} {:<18} {:<18} {:<18} {:<18} {:<18} {:<18} {:<18} {:<18} {:<18}".format(
    "File", "Strategy", "Num_runways", "Exec Time(s)", "Memory Usage", "Status", "Conflicts", "Branches", "Booleans", "Best Obj. Bound", "Variables", "Constraints"
))
for m in all_metrics["CP Multiple"]:
    print("{:<12} {:<18} {:<18.4f} {:<18} {:<18} {:<18} {:<18} {:<18} {:<18} {:<18} {:<18} {:<18}".format(
        m["file"], m["strategy"], m["num_runways"], m["execution_time"], m["memory_usage"],
        m["solution_status"], m["num_conflicts"], m["num_branches"],
        m["num_booleans"], m["best_objective_bound"], m["num_variables"], m["num_constraints"]
    ))


=== Testing strategy: Automatic Search ===


Processing file: airland1.txt


---> Solving with 1 runways...

		     Creating CP model

-> Number of decision variables created: 130
-> Number of constraints: 310

			Solving CP

-> Landing times of all planes:
Plane | Landing Time | Earliest | Target | Latest | Runway
----------------------------------------------------------
    0 |       165.00 |   129.00 | 155.00 | 559.00 |      0
    1 |       258.00 |   195.00 | 258.00 | 744.00 |      0
    2 |        98.00 |    89.00 |  98.00 | 510.00 |      0
    3 |       106.00 |    96.00 | 106.00 | 521.00 |      0
    4 |       118.00 |   110.00 | 123.00 | 555.00 |      0
    5 |       126.00 |   120.00 | 135.00 | 576.00 |      0
    6 |       134.00 |   124.00 | 138.00 | 577.00 |      0
    7 |       142.00 |   126.00 | 140.00 | 573.00 |      0
    8 |       150.00 |   135.00 | 150.00 | 591.00 |      0
    9 |       180.00 |   160.00 | 180.00 | 657.00 |      0

-> Planes that did not land on t

### Large Datasets

In [ ]:
if os.path.exists(output_json):
    with open(output_json, "r") as f:
        all_metrics = json.load(f)
else:
    all_metrics = {}

all_metrics["CP Multiple Large Datasets"] = []

# Values of runways to test for each dataset, based on prior knowledge
runways = {'9': [3,4], '10': [4,5], '11': [5], '12': [6], '13': [7]}

strategies = {
    "Automatic Search": cp_model.AUTOMATIC_SEARCH,
    "Fixed Search": cp_model.FIXED_SEARCH,
    "Portfolio Search": cp_model.PORTFOLIO_SEARCH,
    "LP Search": cp_model.LP_SEARCH
}

for name, strategy in strategies.items():
    print(f"\n=== Testing strategy: {name} ===\n")
    tag = f"CP Multiple Large Datasets - Strategy {name}"

    for i in range(9, 14):
        file_name = f"airland{i}.txt"
        file_path = os.path.join(data_dir, file_name)

        print("\n" + "="*60)
        print(f"Processing file: {file_name}")
        print("="*60 + "\n")

        data = read_airland_file(file_path)

        for r in runways[str(i)]:
            print(f"\n---> Solving with {r} runways...\n")
            separation_times_between_runways = generate_separation_between_runways(data['p'], r, separation_same_runway=True, default_between_runways=2)
            solver, model, variables, metrics = solve_multiple_runways_cp(data['p'], r, data['planes'], data['separation_times'], separation_times_between_runways,
                                                    hint=True, search_strategy=strategy, performance=True)

            penalty = metrics["best_objective_bound"]
            print(f"Runways={r} -> Best Objective Bound={abs(penalty):.2f}")

            save_solution(
                solver = solver,
                variables=variables,
                num_planes=data['p'],
                data=data['planes'],
                solution_file=solutions_json,
                tag="CP Multiple Large Datasets",
                dataset_name=file_name,
                num_runways=r
            )

            metrics_with_file = {"file": file_name, "strategy": name, "num_runways": r, **metrics}
            all_metrics["CP Multiple Large Datasets"].append(metrics_with_file)

            # Interrupt if zero penalty is found
            if penalty == 0:
                break

with open(output_json, "w") as f:
    json.dump(all_metrics, f, indent=4)

print(f"\n-> Metrics saved to {output_json}")

# Metrics Table
print("\nMetrics Table (CP Multiple Large Datasets):")
print("{:<12} {:<18} {:<18} {:<18} {:<18} {:<18} {:<18} {:<18} {:<18} {:<18} {:<18} {:<18}".format(
    "File", "Strategy", "Num_runways", "Exec Time(s)", "Memory Usage", "Status", "Conflicts", "Branches", "Booleans", "Best Obj. Bound", "Variables", "Constraints"
))
for m in all_metrics["CP Multiple Large Datasets"]:
    print("{:<12} {:<18} {:<18.4f} {:<18} {:<18} {:<18} {:<18} {:<18} {:<18} {:<18.2f} {:<18.2f}".format(
        m["file"], m["strategy"], m["num_runways"], m["execution_time"], m["memory_usage"],
        m["solution_status"], m["num_conflicts"], m["num_branches"],
        m["num_booleans"], m["best_objective_bound"], m["num_variables"], m["num_constraints"]
    ))



=== Testing strategy: Automatic Search ===


Processing file: airland9.txt


---> Solving with 3 runways...

		     Creating CP model

-> Number of decision variables created: 10300
-> Number of constraints: 15862

			Solving CP

-> Landing times of all planes:
Plane | Landing Time | Earliest |   Target |   Latest | Runway
--------------------------------------------------------------
    0 |       908.00 |   601.00 |   908.00 |  2401.00 |      0
    1 |      1027.00 |   725.00 |  1027.00 |  2525.00 |      2
    2 |      1378.00 |   928.00 |  1378.00 |  2728.00 |      2
    3 |      1502.00 |   974.00 |  1502.00 |  2774.00 |      2
    4 |      1618.00 |  1056.00 |  1618.00 |  2856.00 |      2
    5 |      1235.00 |  1119.00 |  1235.00 |  2919.00 |      0
    6 |      1520.00 |  1148.00 |  1520.00 |  2948.00 |      0
    7 |      1401.00 |  1195.00 |  1401.00 |  2995.00 |      0
    8 |      1584.00 |  1201.00 |  1584.00 |  3001.00 |      1
    9 |      1343.00 |  1218.00 |  1343.00 |

## Hybrid CP-MIP

In [3]:
if os.path.exists(output_json):
    with open(output_json, "r") as f:
        all_metrics = json.load(f)
else:
    all_metrics = {}

all_metrics["Hybrid"] = []

strategies = {
    "Automatic Search": cp_model.AUTOMATIC_SEARCH,
    "Fixed Search": cp_model.FIXED_SEARCH,
    "Portfolio Search": cp_model.PORTFOLIO_SEARCH,
    "LP Search": cp_model.LP_SEARCH
}

for name, strategy in strategies.items():
    print(f"\n=== Testing strategy: {name} ===\n")
    tag = f"Hybrid - Strategy {name}"

    for i in range(1, num_files + 1):
        file_name = f"airland{i}.txt"
        file_path = os.path.join(data_dir, file_name)

        print("\n" + "="*60)
        print(f"Processing file: {file_name}")
        print("="*60 + "\n")

        data = read_airland_file(file_path)

        for r in range(1, r_max + 1):
            print(f"\n---> Solving with {r} runways...\n")
            separation_times_between_runways = generate_separation_between_runways(data['p'], r, separation_same_runway=True, default_between_runways=2)
            solver, model, fixed_runways, sp_times, metrics = solve_hybrid_lbbd(data['p'], r, data['planes'], data['separation_times'],
                                                                separation_times_between_runways, max_iterations=50, search_strategy=strategy, performance=True)

            penalty = metrics["total_best_objective_bound"]
            print(f"Runways={r} -> Total Penalty={penalty}")

            save_solution(
                solver = None,
                variables=None,
                num_planes=data['p'],
                data=data['planes'],
                solution_file=solutions_json,
                tag=tag,
                dataset_name=file_name,
                num_runways=r,
                landing_times_override=sp_times,
                fixed_runways=fixed_runways
            )

            metrics_with_file = {"file": file_name, "strategy": name, "num_runways": r, **metrics}
            all_metrics["Hybrid"].append(metrics_with_file)

            # Interrupt if zero penalty is found
            if penalty == 0:
                break

with open(output_json, "w") as f:
    json.dump(all_metrics, f, indent=4)

print(f"\n-> Metrics saved to {output_json}")

# Metrics Table
print("\nMetrics Table (Hybrid):")
print("{:<12} {:<18} {:<18} {:<18} {:<18} {:<18} {:<18} {:<18} {:<18} {:<18} {:<18} {:<18} {:<18} {:<18} {:<18} {:<18}".format(
    "File", "Best Obj. Bound", "Strategy", "Num_runways", "Iterations", "Converged", "Exec Time(s)", "CP Time(s)", "MIP Time(s)", "MIP Calls", "Branches", "Conflicts", "Booleans", "Variables", "Constraints", "Memory Usage (MB)"
))
for m in all_metrics["Hybrid"]:
    print("{:<12} {:<18} {:<18} {:<18} {:<18} {:<18} {:<18} {:<18} {:<18} {:<18} {:<18} {:<18} {:<18} {:<18} {:<18} {:<18}".format(
        m["file"], m["total_best_objective_bound"], m["strategy"], m["num_runways"], m["num_iterations"], m["converged"], m["total_time"],
        m["cp_time"], m["mip_time"], m["mip_num_calls"], m["cp_num_branches"], m["cp_num_conflicts"],
        m["cp_num_booleans"], m["cp_num_variables"], m["cp_num_constraints"], m["memory_start_MB"]
    ))


=== Testing strategy: Automatic Search ===


Processing file: airland1.txt


---> Solving with 1 runways...


		Running Hybrid LBBD Solver (Strengthened Master)

--- Iteration 1 ---
  >> Master Theta: 700 | Subproblem Real Cost: 700

*** CONVERGENCE ACHIEVED in 1 iterations! ***

-> Landing times of all planes:
Plane | Landing Time | Earliest | Target | Latest | Runway
----------------------------------------------------------
    0 |       165.00 |   129.00 | 155.00 | 559.00 |      1
    1 |       258.00 |   195.00 | 258.00 | 744.00 |      1
    2 |        98.00 |    89.00 |  98.00 | 510.00 |      1
    3 |       106.00 |    96.00 | 106.00 | 521.00 |      1
    4 |       118.00 |   110.00 | 123.00 | 555.00 |      1
    5 |       126.00 |   120.00 | 135.00 | 576.00 |      1
    6 |       134.00 |   124.00 | 138.00 | 577.00 |      1
    7 |       142.00 |   126.00 | 140.00 | 573.00 |      1
    8 |       150.00 |   135.00 | 150.00 | 591.00 |      1
    9 |       180.00 |   160.00 | 180

### Large Datasets

In [ ]:
if os.path.exists(output_json):
    with open(output_json, "r") as f:
        all_metrics = json.load(f)
else:
    all_metrics = {}

all_metrics["Hybrid Large Datasets"] = []

# Values of runways to test for each dataset, based on prior knowledge
runways = {'9': [3,4], '10': [4,5], '11': [5], '12': [6], '13': [7]}

strategies = {
    "Automatic Search": cp_model.AUTOMATIC_SEARCH,
    "Fixed Search": cp_model.FIXED_SEARCH,
    "Portfolio Search": cp_model.PORTFOLIO_SEARCH,
    "LP Search": cp_model.LP_SEARCH
}

for name, strategy in strategies.items():
    print(f"\n=== Testing strategy: {name} ===\n")
    tag = f"Hybrid Large Datasets - Strategy {name}"

    for i in range(9, 14):
        file_name = f"airland{i}.txt"
        file_path = os.path.join(data_dir, file_name)

        print("\n" + "="*60)
        print(f"Processing file: {file_name}")
        print("="*60 + "\n")

        data = read_airland_file(file_path)

        for r in runways[str(i)]:
            print(f"\n---> Solving with {r} runways...\n")
            separation_times_between_runways = generate_separation_between_runways(data['p'], r, separation_same_runway=True, default_between_runways=2)
            solver, model, sp_times, metrics = solve_hybrid_lbbd(data['p'], r, data['planes'], data['separation_times'],
                                                                separation_times_between_runways, max_iterations=5, search_strategy=strategy, performance=True)

            penalty = metrics["total_best_objective_bound"]
            print(f"Runways={r} -> Total Penalty={abs(penalty):.2f}")

            save_solution(
                solver = None,
                variables=None,
                num_planes=data['p'],
                data=data['planes'],
                solution_file=solutions_json,
                tag=tag,
                dataset_name=file_name,
                num_runways=r,
                landing_times_override=sp_times
            )

            metrics_with_file = {"file": file_name, "strategy": name, "num_runways": r, **metrics}
            all_metrics["Hybrid Large Datasets"].append(metrics_with_file)

            # Interrupt if zero penalty is found
            if penalty == 0:
                break

with open(output_json, "w") as f:
    json.dump(all_metrics, f, indent=4)

print(f"\n-> Metrics saved to {output_json}")

# Metrics Table
print("\nMetrics Table (Hybrid Large Datasets):")
print("{:<12} {:<18} {:<18} {:<18} {:<18} {:<18} {:<18} {:<18} {:<18} {:<18} {:<18} {:<18} {:<18} {:<18} {:<18} {:<18}".format(
    "File", "Best Obj. Bound", "Strategy", "Num_runways", "Iterations", "Converged", "Exec Time(s)", "CP Time(s)", "MIP Time(s)", "MIP Calls", "Branches", "Conflicts", "Booleans", "Variables", "Constraints", "Memory Usage (MB)"
))

for m in all_metrics["Hybrid Large Datasets"]:
    print("{:<12} {:<18} {:<18} {:<18} {:<18} {:<18} {:<18} {:<18} {:<18} {:<18} {:<18} {:<18} {:<18} {:<18} {:<18} {:<18}".format(
        m["file"], m["total_best_objective_bound"], m["strategy"], m["num_runways"], m["num_iterations"], m["converged"], m["total_time"],
        m["cp_time"], m["mip_time"], m["mip_num_calls"], m["cp_num_branches"], m["cp_num_conflicts"],
        m["cp_num_booleans"], m["cp_num_variables"], m["cp_num_constraints"], m["memory_start_MB"]
    ))



=== Testing strategy: Automatic Search ===


Processing file: airland9.txt


---> Solving with 3 runways...


		Running Hybrid LBBD Solver (Strengthened Master)

--- Iteration 1 ---
  >> Master Theta: 65 | Subproblem Real Cost: 78
  >> Gap found. Adding Optimality Cut.
--- Iteration 2 ---
  >> Master Theta: 65 | Subproblem Real Cost: 78
  >> Gap found. Adding Optimality Cut.
--- Iteration 3 ---
  >> Master Theta: 65 | Subproblem Real Cost: 78
  >> Gap found. Adding Optimality Cut.
--- Iteration 4 ---
  >> Master Theta: 65 | Subproblem Real Cost: 78
  >> Gap found. Adding Optimality Cut.
--- Iteration 5 ---
  >> Master Theta: 65 | Subproblem Real Cost: 78
  >> Gap found. Adding Optimality Cut.
Runways=3 -> Total Penalty=77.90

---> Solving with 4 runways...


		Running Hybrid LBBD Solver (Strengthened Master)

--- Iteration 1 ---
  >> Master Theta: 2 | Subproblem Real Cost: 3
  >> Gap found. Adding Optimality Cut.
--- Iteration 2 ---
  >> Master Theta: 2 | Subproblem Real Cost: 3
  >> 